In [1]:
# cell-1  
#load and clean the data (removing diacritics and unwanted text)

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


log_file = 'FloDusTA_1.txt'
with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')




df = pd.read_csv('benchmarks/FloDusTA.csv', encoding='utf-8', engine='python') #, quotechar="'"  , quoting=3
display(df.columns)
df.fillna('', inplace=True)

display(df[:4])


df = df[df['Full_text'] != '']


classes = set(df['Label'].values)
display(classes)

df['label'] = df['Label']


df = df[['Full_text', 'label']]


classes_num = len(classes)
display(classes_num)
display(len(df))

ds = Dataset.from_pandas(df)

ds = ds.train_test_split(test_size=0.2)
display(ds)

max_sequence_length = 128


models = [ 
        'aubmindlab/bert-base-arabertv02-twitter',
        'CAMeL-Lab/bert-base-arabic-camelbert-da',
        'qarib/bert-base-qarib',
        'reemalyami/AraRoBERTa-SA',    
]

for model_name in models:
    for i in range(3):
        print(f'{model_name}, try:{i}')
              
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                              num_labels=classes_num).to('cuda')                                                 
        dataset_train = ds['train']
        dataset_validation = ds['test']                                                    
        
      

        def preprocess_function(examples):
            return tokenizer(examples['Full_text'], truncation=True, padding="max_length",
                            max_length=max_sequence_length, add_special_tokens=True)
        
        
        dataset_train = dataset_train.map(preprocess_function, batched=True)
        dataset_validation = dataset_validation.map(preprocess_function, batched=True)
        
       
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')   
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}


        
        
        epochs = 5
        save_steps = 10000 #save checkpoint every 10000 steps
        batch_size = 64
        
        training_args = TrainingArguments(
            output_dir = 'bert/',
            overwrite_output_dir=True,
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            save_steps = save_steps,
            save_total_limit = 1, #only save the last 5 checkpoints
            fp16=True,
            learning_rate = 5e-5,  # 5e-5 is the default
            logging_steps = 20, #50_000
            evaluation_strategy = 'steps',
            # evaluate_during_training = True,
            eval_steps = 20
            
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            # data_collator=data_collator,
            train_dataset=dataset_train,
            eval_dataset=dataset_validation,
            compute_metrics = compute_metrics
        )
        
        
        # trainer.train(resume_from_checkpoint=True)
        trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv('FloDusTA_results_1.csv')
display(best_results)



2024-03-27 01:37:07.918099: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-27 01:37:07.942961: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 01:37:08.313836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Index(['id_str', 'Full_text', 'Label'], dtype='object')

,id_str,Full_text,Label
0,998294855544590336,هيئة محامي دارفور تدين حادثة اغتيال الطالب عوض الله أبكر وتطالب بعدم إفلات الجناة من المحاكمة https://t.co/a7boK6DHb2,0
1,989053332915474432,#واجب_القراءة للاستاذ جميل الذيابي @JameelAlTheyabi https://t.co/pg9KTlrGKk,0
2,980602110126559232,@M_ALROKIBH السلام عليكم ورحمة الله وبركاته الاخ منصور هل نراك بعد هذا التقرير تصور وانت تقود السيارة او لا مع العلم انك تحذر من استخدام الجوال بتقريرك اثناء القيادة واكثر اسباب حوادث المرورية هو استخدام الجوال .....؟,0
3,999051083438854146,حادثة غريبة على مجتمعنا.\nمن له مصلحة بقتل رجل كهذا؟\nوفي شهر رمضان؟ 🤔\n\n#وفاه_الدكتور_ابراهيم_الغصن,0


{0, 1, 2, 3}

4

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at reemalyami/AraRoBERTa-SA and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7198 [00:00<?, ? examples/s]

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
20,1.069600,0.689214,0.721667,0.616939
40,0.593800,0.392336,0.845556,0.845128
60,0.390400,0.499274,0.822222,0.793439
80,0.363200,0.298558,0.877222,0.877091
100,0.323600,0.260473,0.898889,0.896324
120,0.291600,0.255009,0.902778,0.901374
140,0.201800,0.240046,0.915000,0.910372
160,0.221100,0.259884,0.898333,0.897460
180,0.207900,0.215691,0.917222,0.914050
200,0.203100,0.209722,0.918333,0.914992


,Model,Accuracy,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-da,0.946111,0.944364
3,aubmindlab/bert-base-arabertv02-twitter,0.962222,0.960240
4,qarib/bert-base-qarib,0.956111,0.953769
7,reemalyami/AraRoBERTa-SA,0.933333,0.929994
